Both!

In [ ]:
import numpy as np
import kepler
import Quaternion
from scipy import integrate
import matplotlib.pyplot as plt

%matplotlib inline

the spin rate of the earth changes too
start off assuming steady state, that the moon's angular velocities in the inertial frame and the earth's angular velocities in the inertial frame are the same, so in a little time dt, the bulge and moon maintain the same seperation.

Then use change in phi* and theta* for the moon to calculate its new angular velocity about the earth.
Then use conservation of angular momentum to calculate the earth's new angular velocity. 
delta is then equal to delta + or - the little angle rotated by the moon minus the little angle rotated by the earth

delta = delta0 + (omegaMoon * t - omegaEarth * t)

In the even more general case, model the evolution tidal deformation as a satellite starts off far away and comes closer. rederive the formulas, i think. 

*Implement by 1/17/2026

**Check equations with Dr. Johnston 

In [ ]:
def findInertiaTensor(a, b, c):
    #inertia tensor over a,b,c
    tensor = np.zeros((3,3))
    #closed form expression

def findScaledInertiaTensor(a, b, c):
    tensor = np.zeros((3,3))
    #by symmetry, off-diagonals vanich
    # Tij = ri * rj * ||r||^2

def rotateToBodyFrame(r, state):
    q = Quaternion.Quaternion(state[5], -state[6:9]) #Encodes orientation of satellite frame wrt inertial frame
    rotated_bodyframe_r = (q.inverse()*Quaternion.Quaternion(0, [-r[0], -r[1], -r[2]])*q)[1:4] #rotates from global frame to local frame. 
    return rotated_bodyframe_r

def findTorqueVector(state, GAMMA_R_CUBED, InertiaTensor, ScaledInertiaTensor, DELTA, a, b, c): 
    Rcm = rotateToBodyFrame(np.array([state[0]*np.sin(state[1])*np.cos(state[2]), state[0]*np.sin(state[1])*np.cos(state[2]), state[0]*np.cos(state[1])]), state)
    thetacm = rotateToBodyFrame(np.array([state[0]*np.sin(state[1]-np.pi/2)*np.cos(state[2]), state[0]*np.sin(state[1]-np.pi/2)*np.cos(state[2]), state[0]*np.cos(state[1]-np.pi/2)]), state)
    gamma = GAMMA_R_CUBED / state[0]**3
    P2 = 3 * (np.cos(state[1] - DELTA))**2 - 1
    kappa = np.sin(2*state[1] - 2 * DELTA)

    torque1 = (3 * MU / state[0]**5) * np.cross(Rcm, InertiaTensor @ Rcm)
    torque2 = (5 / state[0]**7) * np.cross(Rcm, InertiaTensor @ Rcm)
    torque3 = (35 / (2 * state[0]**9)) * np.cross(Rcm, ScaledInertiaTensor @ Rcm)
    torque4 = (-5 / state[0]**7) * np.cross(thetacm, InertiaTensor @ Rcm)
    torque5 = (-(a**2 + b**2 + c**2) / state[0]**7) * np.cross(thetacm, Rcm)
    torque6 = (35 / (2 * state[0]**9)) * np.cross(thetacm, ScaledInertiaTensor @ Rcm)

    return torque1 + gamma * P2 * (torque2 + torque3) + gamma * kappa * (torque4 + torque5 + torque6)

#body frame alligned so I_x axis, I_y axis, I_z axis
# (x-x0)^2 / a^2 + (y-y0)^2 / b^2 + (z-z0)^2 / c^2 = 1 // a > b > c so Ix < Iy < Iz

#state = [r1, r1dot theta, thetadot, phi (azimuthal), phidot, w1 moon, w2 moon, w3 moon, q0, q1, q2, q3, w1 earth, w2 earth, w3 earth]
def ODE(state, t, G, M, m, GAMMA_R_CUBED, IxxSecondary, IyySecondary, IzzSecondary, IxxPrimary, IyyPrimary, IzzPrimary):

    torque = findTorqueVector(state, GAMMA_R_CUBED, InertiaTensor, ScaledInertiaTensor, DELTA, IxxSecondary, IyySecondary, IzzSecondary)
    #Euler's Equations with torque follow quite directly from Lagrangian Mechanics

    rd = state[1]
    rdd = state[0] * state[3]**2 - (G*M) / state[0]**2 - (GAMMA_R_CUBED / state[0]**7) * (3 * np.cos(state[2] - DELTA)**2 - 1)
    thetad = state[3]
    thetadd = -2 * state[1] * state[3] / state[0] - (GAMMA_R_CUBED / state[0]**8) * np.sin(2*(state[2] - DELTA))
    phid = state[5]
    phidd = -2 * state[1] * state[5] / state[0]

    #subrtacction -- check signage on torque
    omega_dot_1 = ((IyySecondary-IzzSecondary) * state[7] * state[8] + torque[0]) / IxxSecondary
    omega_dot_2 = ((IzzSecondary-IxxSecondary) * state[8] * state[6] + torque[1]) / IxxSecondary
    omega_dot_3 = ((IxxSecondary-IyySecondary) * state[6] * state[7] + torque[2]) / IzzSecondary

    #These are relatively simple too. Find the relationship between the time derivative of a quaternion and the angular velocity
    #then carry out quaternion multiplication and equation the components. 
    q_dot_0 = -0.5 * (state[4] * state[0] + state[5] * state[1] + state[6]*state[2])
    q_dot_1 = 0.5 * (state[3] * state[0] - state[6] * state[1] + state[5]*state[2])
    q_dot_2 = 0.5 * (state[6] * state[0] + state[3] * state[1] - state[4]*state[2])
    q_dot_3 = 0.5 * (state[5] * state[0] + state[4] * state[1] - state[3]*state[2])

    #primary's rotation
    omega_dot_1_primary = ((IyyPrimary-IzzPrimary) * state[7] * state[8] + TORQUE[0]) / IxxPrimary
    omega_dot_2_primary = ((IzzPrimary-IxxPrimary) * state[8] * state[6] + TORQUE[1]) / IyyPrimary
    omega_dot_3_primary = ((IxxPrimary-IyyPrimary) * state[6] * state[7] + TORQUE[2]) / IzzPrimary

    #Earth's orientation relative to the inertial frame, alligned so delta is theta = 0. rotate Rcm vector into this frame 
    #to find angle between delta and rcm for Delta or smth
    #torque exerted on primary should have a simple formula
    q_dot_0_primary = -0.5 * (state[4] * state[0] + state[5] * state[1] + state[6]*state[2])
    q_dot_1_pimary = 0.5 * (state[3] * state[0] - state[6] * state[1] + state[5]*state[2])
    q_dot_2_primary = 0.5 * (state[6] * state[0] + state[3] * state[1] - state[4]*state[2])
    q_dot_3_primary = 0.5 * (state[5] * state[0] + state[4] * state[1] - state[3]*state[2])
    return [rd, rdd, thetad, thetadd, phid, phidd, omega_dot_1, omega_dot_2, omega_dot_3, q_dot_0, q_dot_1, q_dot_2, q_dot_3]
